## leer dos archivos y concatenarlos al mismo tiempo

In [1]:
import boto3
import pandas as pd
from io import StringIO

# Inicializar el recurso S3 y el bucket
s3 = boto3.resource('s3')
bucket_name = 'xetra-1234'
bucket = s3.Bucket(bucket_name)

# Filtrar objetos por fecha
prefixes = ['2022-03-15', '2022-03-16']
objects = []
for prefix in prefixes:
    bucket_objs = bucket.objects.filter(Prefix=prefix)
    objects.extend(bucket_objs)

# Leer y concatenar los archivos CSV en un solo DataFrame
dfs = []
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)


C:\Users\JOSUE\AppData\Local\Temp\ipykernel_6604\1265072653.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat(dfs, ignore_index=True)


In [2]:
print(df_all.columns)

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')


In [3]:
columns=['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]

In [4]:
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,DE000A0DJ6J9,2022-03-15,08:00,37.34,37.76,37.30,37.50,11421
1,DE000A0D6554,2022-03-15,08:00,16.58,16.58,16.43,16.45,21537
2,DE000A0D9PT0,2022-03-15,08:00,201.10,202.40,201.00,202.40,1585
3,DE000A0JL9W6,2022-03-15,08:00,62.55,63.20,62.55,63.20,1539
4,DE000A0LD6E6,2022-03-15,08:00,61.40,61.40,60.95,61.05,329
...,...,...,...,...,...,...,...,...
251797,US02005N1000,2022-03-16,20:30,37.80,37.80,37.80,37.80,0
251798,US2533931026,2022-03-16,20:30,101.00,101.00,101.00,101.00,0
251799,US89832Q1094,2022-03-16,20:30,52.50,52.50,52.50,52.50,0
251800,GB00BNYK8G86,2022-03-16,20:30,5.20,5.20,5.20,5.20,0
